In [ ]:
# Plots for section 5.1

### Imports

In [ ]:
import pandas as pd
import matplotlib
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
from datetime import datetime
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
import seaborn as sns
import matplotlib.ticker as mtick


# imported own functions
from utils import  nodal_balance,  get_condense_sum

# imported own definitions
from utils import carrier_colors, carrier_renaming, resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal
from utils import c_h2_gen, c_h2_con, c_el_gen_s, c_el_con_s

# general variables
font1 = {'fontname':'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/5.1_hydrogen_overview'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

In [ ]:
# Network imports
stst =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp =pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

In [ ]:
# stst and exp dataframes
# spatial
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

# temporal
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [ ]:
# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger", "solar thermal"]

In [ ]:
# Notebook Functions

def get_df(df_no, df_h2, carriers):
    result = pd.DataFrame(index = ["STST", "EXP"])

    for c in carriers:
        result.loc["STST" , c] = df_no[c].values
        result.loc["EXP" , c] = df_h2[c].values
    return result

def c_gen(carrier_list, network):
    result = []
    for c in carrier_list:
        if c in network.generators.carrier.unique().tolist():
            result.append(c)
    return result

def c_link(carrier_list, network):
    result = []
    for c in carrier_list:
        if c in network.links.carrier.unique().tolist():
            result.append(c)
    return result

def c_su(carrier_list, network):
    result = []
    for c in carrier_list:
        if c in network.storage_units.carrier.unique().tolist():
            result.append(c)
    return result

In [ ]:
# Regions
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

### Demand

In [ ]:
# loads are the same for exp and stst

In [ ]:
# temporal
loads = ["land transport fuel cell", "H2 for industry", "H2 for shipping"]
stst_loads_ts = stst.loads_t.p.loc[:, stst.loads.carrier.isin(loads)].sum(axis=1) / 1000 * 3

fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2,  height_ratios=[1.5, 1],
                        figsize=(14, 7))

# whole year
ax1.plot(stst_loads_ts, color="turquoise", lw=0.75, label="overall demand")
ax1.plot(stst_loads_ts.resample("D").mean(), color="black", lw=1, label="daily mean")
#ax1.plot(stst_loads_ts.resample("7D").mean(), color="red", lw=1, label="weekly mean")
ax1.set_title(f"Hydrogen demand (all year)", fontsize=16, **font1)
ax1.set_ylabel("GWh")
ax1.legend(loc="upper left")

# January
ax2.plot(stst_loads_ts["2013-03-4":"2013-03-17"], color="turquoise")
ax2.set_title(f"Hydrogen demand of two weeks in March (2013-03-4 until 2013-03-17)", fontsize=16, **font1)
ax2.set_ylabel("GWh")
ax2.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%a %d-%b'))

fig.tight_layout(pad=1)
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_demand_temporal.png")

In [ ]:
# spatial
n = stst
stst_loads_sp = n.loads_t.p.loc[:, n.loads.carrier.isin(loads)].sum(axis=0) / 1000 / 1000 * 3
stst_loads_sp.index = stst_loads_sp.index.map(n.loads.bus)
stst_loads_sp.index = stst_loads_sp.index.map(n.buses.location)
stst_loads_sp = stst_loads_sp.groupby(stst_loads_sp.index, axis=0).sum()
df_stst_ons["h2_demand"] = stst_loads_sp

# load per carrier
for c in loads:

    n = stst
    stst_loads_sp = n.loads_t.p.loc[:, n.loads.carrier == c].sum(axis=0) / 1000 / 1000 * 3
    stst_loads_sp.index = stst_loads_sp.index.map(n.loads.bus)
    stst_loads_sp.index = stst_loads_sp.index.map(n.buses.location)
    stst_loads_sp = stst_loads_sp.groupby(stst_loads_sp.index, axis=0).sum()
    df_stst_ons[f"h2_demand_{c}"] = stst_loads_sp

In [ ]:
df = df_stst_ons

fig, ax = plt.subplots(ncols=1, nrows=1,
                       subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(8, 7))
crs = ccrs.EqualEarth()


ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')


df.to_crs(crs.proj4_init).plot(column="h2_demand",
                                               ax=ax,
                                               cmap=plt.get_cmap('spring_r'),
                                               linewidth=0.05,
                                               edgecolor = 'grey',
                                               legend=True,
                                               legend_kwds={'label':"Hydrogen demand [($TWh_{h2}$)]",
                                                            'orientation': "vertical",
                                                            'shrink' : 0.9}
                               )

ax.set_title(f"Hydrogen demand", fontsize=16, **font1)

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_demand_spatial.png")

In [ ]:
# overall demand (TWh)
stst.loads_t.p.loc[:, stst.loads.carrier.isin(loads)].sum().sum() / 1e6 * 3

In [ ]:
# demand per region
df_stst_ons["h2_demand"].sort_values(ascending=False)

In [ ]:
# subdivision
# "land transport fuel cell", "H2 for industry", "H2 for shipping"
stst.loads_t.p.loc[:, stst.loads.carrier == "H2 for industry"].sum().sum() / 1e6 * 3

In [ ]:
df_stst_ons["h2_demand"][df_stst_ons["h2_demand"] > 10].sum()#.sort_values(ascending=False)

In [ ]:
carriers = ["h2_demand"] + ["h2_demand_" + l for l in loads]
df = df_stst_ons

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 14))
crs = ccrs.EqualEarth()


for i, ax in enumerate(axs.reshape(-1)):

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}",
                                                   ax=ax,
                                                   cmap=plt.get_cmap('spring_r'),
                                                   linewidth=0.05,
                                                   edgecolor = 'grey',
                                                   legend=True,
                                                   legend_kwds={'label':"Electricity balance [TWh]",
                            'orientation': "vertical",
                                      'shrink' : 0.8})


    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)

fig.tight_layout()
plt.show()

In [ ]:
#df_stst_ons["h2_demand_H2 for industry"].sort_values(ascending=False)
df_stst_ons["h2_demand_H2 for shipping"].sort_values(ascending=False)
#df_stst_ons["h2_demand_land transport fuel cell"].sort_values(ascending=False)

### Network & Flow

In [ ]:
l_h2 = exp.copy()

locs = l_h2.buses[l_h2.buses.carrier == "AC"][["x","y"]]
mapping = pd.DataFrame(l_h2.buses[l_h2.buses.carrier == "H2"].location)
mapping["bus"] = mapping.index
mapping.set_index("location", inplace =True)
locs.index = locs.index.map(mapping.to_dict()['bus'])
l_h2.buses.x[l_h2.buses.carrier == "H2"] = locs.x
l_h2.buses.y[l_h2.buses.carrier == "H2"] = locs.y

l_h2.mremove("Bus",l_h2.buses[l_h2.buses.carrier != "H2"].index)

# write LineStrign into pipes (links)
h2_pipes = l_h2.links[l_h2.links.carrier.isin(["H2 pipeline retrofitted" , "H2 pipeline"])].index
other_links = l_h2.links[-l_h2.links.carrier.isin(["H2 pipeline retrofitted" , "H2 pipeline"])].index

for pipe in h2_pipes:
    loc1 = l_h2.buses.loc[l_h2.links.loc[pipe].bus0][["x", "y"]]
    loc2 = l_h2.buses.loc[l_h2.links.loc[pipe].bus1][["x", "y"]]
    l_h2.links.geometry.loc[pipe] = LineString([Point(loc1), Point(loc2)]).wkt

l_h2.mremove("Link", other_links)
l_h2.mremove("Line", l_h2.lines.index)

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(8, 6))

l_h2.plot(ax=ax, link_colors="pink", link_widths=0.8,  projection=ccrs.EqualEarth())
pypsa.plot.add_legend_patches(ax=ax, labels=["hydrogen pipes"], colors=["pink"], legend_kw={'loc': 'upper left'})
ax.set_title("H2 network", pad=20)
plt.show()

In [ ]:
# adding colorbar to this

fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(8, 6))

l_h2.links.p_max_pu
l_h2.plot(ax=ax, link_colors=l_h2.links.p_nom_opt, link_cmap=plt.get_cmap("magma_r"), link_widths=np.log10(l_h2.links.p_nom_opt)/2,  projection=ccrs.EqualEarth())
ax.set_title("H2 network", pad=10)
plt.show()

In [ ]:
# with colorbar
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(8, 6))

# GW
link_loading = l_h2.links.p_nom_opt / 1000

cmap= plt.cm.inferno_r
norm = mcolors.Normalize(vmin=link_loading.min(), vmax=link_loading.max())
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
colors = list(map(mcolors.to_hex, cmap(norm(link_loading))))

l_h2.plot(ax=ax, link_colors=colors, link_widths=2,  projection=ccrs.EqualEarth())
plt.colorbar(sm, orientation='vertical', shrink=0.9, ax=ax, label="Capacity in GW")
plt.title("Hydrogen network (EXP)", pad=10)

plt.show()

In [ ]:
# hydrogen pipeline network in km
exp.links[exp.links.carrier.isin(["H2 pipeline retrofitted" , "H2 pipeline"])].length.sum()

In [ ]:
# hydrogen network in TW*km

In [ ]:
((exp.links[exp.links.carrier.isin(["H2 pipeline retrofitted" , "H2 pipeline"])].p_nom_opt * exp.links[exp.links.carrier.isin(["H2 pipeline retrofitted" , "H2 pipeline"])].length) / 1e6).sum()

In [ ]:
# pipeline capacities
link_loading.sort_values(ascending=False).mean()

In [ ]:
# Balance Flow

In [ ]:
h2_balance = nodal_balance(exp, "H2", aggregate=["snapshot"])

# change sign so that exports are positive and imports negative (TWh)
df_exp_ons["h2_balance"] = -(h2_balance.droplevel(0).unstack("carrier").filter(like="H2 pipeline").sum(axis=1)).div(1e6)

In [ ]:
# nodal balance electricity and hydrogen
df = df_exp_ons

fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, 6))
crs = ccrs.EqualEarth()

ax1.add_feature(cartopy.feature.OCEAN, color='azure')

# GW
link_loading = l_h2.links.p_nom_opt / 1000

cmap= plt.cm.inferno_r
norm = mcolors.Normalize(vmin=link_loading.min(), vmax=link_loading.max())
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
colors = list(map(mcolors.to_hex, cmap(norm(link_loading))))

l_h2.plot(ax=ax1, link_colors=colors, link_widths=2,  projection=ccrs.EqualEarth())
plt.colorbar(sm, orientation='vertical', shrink=0.9, ax=ax1, label="Capacity in GW")
ax1.set_title("Hydrogen network (EXP)", pad=10)

ax2.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax2.coastlines(edgecolor='black', linewidth=0.5)
ax2.set_facecolor('white')
ax2.add_feature(cartopy.feature.OCEAN, color='azure')

abs_max = max(abs(df["h2_balance"].max()) , abs(df["h2_balance"].min()))

df.to_crs(crs.proj4_init).plot(column="h2_balance",
                                               ax=ax2,
                                               cmap=plt.get_cmap('PuOr'),
                                               vmax=abs_max,
                                               vmin=-abs_max,
                                               linewidth=0.05,
                                               edgecolor = 'grey',
                                               legend=True,
                                               legend_kwds={'label':"Hydrogen balance [TWh]",
                                                            'orientation': "vertical",
                                                            'shrink' : 0.9}
                               )

ax2.set_title(f"Hydrogen balance (EXP)", fontsize=22, **font1)

l_h2.plot(ax=ax2, link_colors="indigo", link_widths=2e-5,  flow="sum", projection=ccrs.EqualEarth())
pypsa.plot.add_legend_patches(ax=ax2, labels=["hydrogen pipes and flow direction"], colors=["indigo"], legend_kw={'loc': 'upper left'})
ax2.set_title("Hydrogen balance and flow (EXP)", pad=10)

fig.tight_layout()
plt.show()
#fig.savefig(f"{PLOT_DIR}/h2_network+flow_EXP.png")

In [ ]:
# balance regions (TWh)
df_exp_ons["h2_balance"].sort_values(ascending=False)

In [ ]:
# flows
(exp.links_t.p0.loc[: , h2_pipes].sum() / 1e6 *3).sort_values()

In [ ]:
l_h2 = exp.copy()

locs = l_h2.buses[l_h2.buses.carrier == "AC"][["x","y"]]
mapping = pd.DataFrame(l_h2.buses[l_h2.buses.carrier == "H2"].location)
mapping["bus"] = mapping.index
mapping.set_index("location", inplace =True)
locs.index = locs.index.map(mapping.to_dict()['bus'])
l_h2.buses.x[l_h2.buses.carrier == "H2"] = locs.x
l_h2.buses.y[l_h2.buses.carrier == "H2"] = locs.y

l_h2.mremove("Bus",l_h2.buses[l_h2.buses.carrier != "H2"].index)

# write LineStrign into pipes (links)
h2_pipes = l_h2.links[l_h2.links.carrier.isin(["H2 pipeline retrofitted" , "H2 pipeline"])].index
other_links = l_h2.links[-l_h2.links.carrier.isin(["H2 pipeline retrofitted" , "H2 pipeline"])].index

for pipe in h2_pipes:
    loc1 = l_h2.buses.loc[l_h2.links.loc[pipe].bus0][["x", "y"]]
    loc2 = l_h2.buses.loc[l_h2.links.loc[pipe].bus1][["x", "y"]]
    l_h2.links.geometry.loc[pipe] = LineString([Point(loc1), Point(loc2)]).wkt

l_h2.mremove("Link", other_links)
l_h2.mremove("Line", l_h2.lines.index)

In [ ]:
# Hydrogen network
# link_cmap=plt.get_cmap("magma_r")

fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(8, 8))

l_h2.plot(ax=ax, link_colors="pink", link_widths=2e-5,  flow="sum", projection=ccrs.EqualEarth())
pypsa.plot.add_legend_patches(ax=ax, labels=["hydrogen pipes"], colors=["pink"], legend_kw={'loc': 'upper left'})
ax.set_title("H2 network", pad=20)
plt.show()

### Capacity

In [ ]:
# new calculated cap
cap_stst_sum_gen = pd.DataFrame(df_stst_ons[df_stst_ons.columns[df_stst_ons.columns.str.contains("cap_gen_h2")]].sum()).transpose()
cap_exp_sum_gen = pd.DataFrame(df_exp_ons[df_exp_ons.columns[df_exp_ons.columns.str.contains("cap_gen_h2")]].sum()).transpose()

cap_stst_sum_con = pd.DataFrame(df_stst_ons[df_stst_ons.columns[df_stst_ons.columns.str.contains("cap_con_h2")]].sum()).transpose()
cap_exp_sum_con = pd.DataFrame(df_exp_ons[df_exp_ons.columns[df_exp_ons.columns.str.contains("cap_con_h2")]].sum()).transpose()

# calc capacities which are present at EU bus (df_CALC does not map)
cap_stst_sum_con["Fischer-Tropsch_cap_con_h2"] = stst.links[stst.links.carrier == "Fischer-Tropsch"].p_nom_opt.sum() / 1000
cap_exp_sum_con["Fischer-Tropsch_cap_con_h2"] = exp.links[exp.links.carrier == "Fischer-Tropsch"].p_nom_opt.sum() / 1000
cap_stst_sum_con["Sabatier_cap_con_h2"] = stst.links[stst.links.carrier == "Sabatier"].p_nom_opt.sum() / 1000
cap_exp_sum_con["Sabatier_cap_con_h2"] = exp.links[exp.links.carrier == "Sabatier"].p_nom_opt.sum() / 1000

In [ ]:
# capacity of electricity producing technologies

fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=4, figsize=(12, 6))
bbox = (0.5, 1.25)

carriers = ["H2 Electrolysis"]
labels = [c + "_cap_gen_h2" for c in carriers]
get_df(cap_stst_sum_gen, cap_exp_sum_gen, labels).plot(ax = ax0, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in carriers], ylabel="Capacity [GW]")
ax0.legend(labels = carriers, loc='upper center', bbox_to_anchor=bbox)

carriers = ["SMR CC", "SMR"]
labels = [c + "_cap_gen_h2" for c in carriers]
get_df(cap_stst_sum_gen, cap_exp_sum_gen, labels).plot(ax = ax1, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in carriers], ylabel="Capacity [GW]")
ax1.legend(labels = carriers, loc='upper center', bbox_to_anchor=bbox)

carriers = ["Sabatier", "Fischer-Tropsch"]
labels = [c + "_cap_con_h2" for c in carriers]
get_df(cap_stst_sum_con, cap_exp_sum_con, labels).plot(ax = ax2, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in carriers], ylabel="Capacity [GW]")
ax2.legend(labels = carriers, loc='upper center', bbox_to_anchor=bbox)

carriers = [ "H2 liquefaction", "H2 Fuel Cell"]
labels = [c + "_cap_con_h2" for c in carriers]
get_df(cap_stst_sum_con, cap_exp_sum_con, labels).plot(ax = ax3, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in carriers], ylabel="Capacity [GW]")
ax3.legend(labels = [carrier_renaming.get(n, n) for n in carriers], loc='upper center', bbox_to_anchor=bbox)

fig.suptitle("Capacity of hydrogen producing and consuming technologies", fontsize=16,  **font1)
fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_gen_con_capa_scenarios.png")

In [ ]:
#cap_stst_sum_gen
#cap_exp_sum_gen
#cap_stst_sum_con
#cap_exp_sum_con

In [ ]:
df_stst_ons["Fischer-Tropsch_cap_con_h2_manual"] = stst.links[stst.links.carrier == "Fischer-Tropsch"].p_nom_opt.values / 1000
df_stst_ons["Sabatier_cap_con_h2_manual"] = stst.links[stst.links.carrier == "Sabatier"].p_nom_opt.values / 1000
df_exp_ons["Fischer-Tropsch_cap_con_h2_manual"] = exp.links[exp.links.carrier == "Fischer-Tropsch"].p_nom_opt.values / 1000
df_exp_ons["Sabatier_cap_con_h2_manual"] = exp.links[exp.links.carrier == "Sabatier"].p_nom_opt.values / 1000

In [ ]:
# Capacity correlation (spatial)
# correlation with all generators, links, su that have production or consumption for heat, el, h2, oil and gas or biom

In [ ]:
cap_cols = df_stst_ons.columns[df_stst_ons.columns.str.contains("cap")]
cap_gen_cols = [c + "_cap_gen_h2" for c in c_h2_gen]
cap_con_cols = ["Fischer-Tropsch_cap_con_h2_manual" , "Sabatier_cap_con_h2_manual", "H2 liquefaction_cap_con_h2"]
df_stst_ons[cap_cols].corr().loc[cap_gen_cols + cap_con_cols]

**Production**

|technology   |STST   |EXP   |
|---|---|---|
|'electrolysis'   |878   |736   |
|'SMR CC'   | 63  | 0.0024  |
| 'SMR'  | 1.4   | 0.0026  |

**Consumption**

|technology   |STST   |EXP   |
|---|---|---|
|'Sabatier'   |287   |22   |
|'Fischer Tropsch'   | 187  | 167  |
| 'H2 liquefaction'  |57   | 57  |
|  'H2 Fuell Cell' | 1.11  | 0.0052  |

## Correlation of Capacity (spatial)

In [ ]:
# pretend capacities of Fischer Tropsch and Sabatier are not all at EU
df_stst_ons["Fischer-Tropsch_cap_con_h2_manual"] = stst.links[stst.links.carrier == "Fischer-Tropsch"].p_nom_opt.values / 1000
df_stst_ons["Sabatier_cap_con_h2_manual"] = stst.links[stst.links.carrier == "Sabatier"].p_nom_opt.values / 1000
df_exp_ons["Fischer-Tropsch_cap_con_h2_manual"] = exp.links[exp.links.carrier == "Fischer-Tropsch"].p_nom_opt.values / 1000
df_exp_ons["Sabatier_cap_con_h2_manual"] = exp.links[exp.links.carrier == "Sabatier"].p_nom_opt.values / 1000

# Capacity correlation (spatial)
# correlation with all generators, links, su that have production or consumption for heat, el, h2, oil and gas or biom
cap_cols = df_stst_ons.columns[df_stst_ons.columns.str.contains("cap")]
cap_gen_cols = [c + "_cap_gen_h2" for c in c_h2_gen]
cap_con_cols = ["Fischer-Tropsch_cap_con_h2_manual", "Sabatier_cap_con_h2_manual", "H2 liquefaction_cap_con_h2"]
df_stst_ons[cap_cols].corr().loc[cap_gen_cols + cap_con_cols]

In [ ]:
df_stst_ons[cap_cols].corr().loc["H2 Fuel Cell_cap_con_h2"].dropna().sort_values().tail(30)
#df_exp_ons[cap_cols].corr().loc["H2 liquefaction_cap_con_h2"].dropna().sort_values().tail(50)

In [ ]:
df_stst_ons["Fischer-Tropsch_cap_con_h2_manual"].corr(df_stst_ons["Sabatier_cap_con_h2_manual"])

**Findings: correlation of capacities amogn the scenarios**

**"H2 Electrolysis_cap_gen_h2**:
STST+neg: high negative correlation with urban central biomass CHP cap con biom (-0.47) and urban central gas CHP cap con gas (-0.34)
STST+pos: Fischer-Tropsch_cap_con_h2_manual (0.94), Sabatier_cap_con_h2_manual (0.68), onwind_cap_gen_el (0.67),
EXP+neg: urban central gas CHP CC_cap_con_gas (-0.44), H2 Fuel Cell_cap (-0-42), SMR CC_cap (-0.40)
EXP+pos: onwind_cap_gen_el (0.76), offwind-dc_cap_gen_el (0.46), solar_cap_gen_el (0.37)

**SMR CC**:
STST+neg: nope
STST+pos: H2 liquefaction (0.72), gas boilder (0.64), gas CHP (0.55), offwind-dc (0.54)

**Fischer-Tropsch**
STST+neg:
STST+pos: electrolysis, onwind, Sabatier (0.55)
EXP+neg:
EXP+pos: heat pump (0.76), resisitive heater (0.73), gas CHP (0.60)

**Sabatier**
STST+neg: -
STST+pos: onwind (0.72), electrolysis (0.68), Fischer (0.55)
EXP+neg: -
EXP+pos: -

**H2 liquefaction**
STST+neg: -
STST+pos: SMR CC (0.72), offwind-dc (0.61)
EXP+neg: -
EXP+pos: gas CHP (0.42)

**H2 Fuel Cell**
STST+neg: -
STST+pos:


Interesting and making sense:
- H2 Electrolysis and onwind: (no: 0.67; h2: 0.76) -> onshore wind dominantly used for hydrogen production
- H2 Electrolysis and solar: (no: 0.32; h2: 0.37) -> solar wind partly used for hydrogen production
- H2 Electrolysis and H2 pipeline: 0.66 -> hydrogen production at locations with pipelines
- onwind and h2 pipeline. 0.57 -> onwind used for hydrogen production close to h2 pipelines; pipelines are build where a lot of onwind is located
- offwind-dc and H2 liquefaction: 0.61 ; 0.35 -> offshore wind used to produced liquefied hydrogen (more present in STST scenario as there is less possibility to transport away the offshore wind power generation)

- solar and battery charger: (no: 0.59; h2: 0.54) -> solar used for utility scale battery charging
- BEV and solar rooftop: (STST: 0.63 ; EXP: 0.56) -> solar used to charge vehicles
- BEV and solar: (STST: 0.53 ; EXP: 0.22) -> solar used to charge vehicles

- gas CHP and heat pump / gas boilder / resistive heater : (no: 0.83, 0.72, 0.72 ; h2: 0.71 , 0.7, 0.58) -> all heat sources are at the same location
- gas CHP and biomass CHP / biomas CHP CC: 0.67 / 0.67 ; 0.44 / 0.3 -> all heat sources are at the same location
- gas CHP and water tanks charger: 0.66 ; 0.53 -> heat used for charging thermal storage
- water tanks charger and heat pump / gas boiler / resisitive heater: 0.68 / 0.65 / 0.67 ; 0.73/ 0.72 / 0.65 -> heat used for charging thermal storage

Not making sense or not interesting?:
- biomas CHP and water tanks charger / discharger; (no: 0.007/ 0.048 ; h2: 0.59, 0.55) -> Why no correlation in STST case?
- biomass CHP and home battery charger: -0.52 ; -0.23
- gas CHP CC and home battery charger: -0.65 ; -0.7
- H2 Electrolysis and water tanks charger / water tanks discharger: ( no: 0.39 / 0.44 ; -0.12 / < abs(0.1)) -> rather random?

Not interesting and making sense:
- solar and solar rooftop (0.5 ; 0.47)
- solar thermal and solar: 0.56 ; 0.41

Not interesting and not making sense:
- H2 Fuel cell and home battery charger / home battery discharger: (no: -0.032; h2: -0.69) not very relevant as there is almost no capacity of fuel cell
- SMR CC and H2 liquefaction: (no: 0.72; h2: -0.16) strong correlation with SMR CC and H2 liquefaction in STST scenario; small correlation in EXP scenario random due to little capacity of SMR CC
- H2 Fuel Cell and biomass CHP / biomass CHP CC: (no: 0.032 / <0.01 ; ) H2 Fuel Cell has no relevance

## Production and Consumption

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 11))

for n in [stst, exp]:

    carrier = ["H2"]
    nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
    # convert from MW to GW
    nb = nb.unstack(level=[1]) / 1e3

    nb_pos = nb.sum()[nb.sum() > 0].sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_pos = nb_pos[(nb_pos / nb_pos.sum()) > 0.01]

    nb_neg = abs(nb.sum()[nb.sum() < 0]).sort_values(ascending=False)
    # exclude all shares smaller than 1 %
    nb_neg = nb_neg[(nb_neg / nb_neg.sum()) > 0.01]

    if n==stst:
        ax_gen=axs[0, 0]
        ax_con=axs[0, 1]
        title_gen=f"Hydrogen production (STST)"
        title_con=f"Hydrogen consumption (STST)"

    elif n==exp:
        ax_gen=axs[1, 0]
        ax_con=axs[1, 1]
        title_gen=f"Hydrogen production (EXP)"
        title_con=f"Hydrogen consumption (EXP)"

    # production
    c = [carrier_colors[col] for col in nb_pos.index]
    percents = nb_pos.to_numpy() * 100 / nb_pos.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_pos.index, percents)]

    patches, texts = ax_gen.pie(nb_pos, colors=c, startangle=0, labels=labels)
    ax_gen.axis('equal')
    ax_gen.set_title(title_gen, pad=20, fontweight="bold")

    # consumption
    c = [carrier_colors[col] for col in nb_neg.index]
    percents = nb_neg.to_numpy() * 100 / nb_neg.to_numpy().sum()
    labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_neg.index, percents)]

    patches, texts = ax_con.pie(nb_neg, colors=c, startangle=0, labels=labels)
    ax_con.axis('equal')

    ax_con.set_title(title_con, pad=20, fontweight="bold")

fig.tight_layout(pad=2)
plt.show()

fig.savefig(f"{PLOT_DIR}/gen_con_h2.png")

In [ ]:
nb

In [ ]:
# where is H2 for shipping?
# This is exaclty the amount that goes into the H2 liquefaction

In [ ]:
#df_stst_ons["H2 liquefaction_con_h2"].sum() /1e3
df_exp_ons["H2 liquefaction_con_h2"].sum() /1e3

In [ ]:
df_stst_ons["H2 liquefaction_gen_h2"].sum() /1e3
df_exp_ons["H2 liquefaction_gen_h2"].sum() /1e3

In [ ]:
# overall production

model = "STST"

if model == "STST":
    n = stst
    index = c_h2_gen

elif model == "EXP":
    n = exp
    index = ["H2 Electrolysis"]

carrier = ["H2"]
nb = nodal_balance(n, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
# convert from MWh to GWh
nb = nb.unstack(level=[1]) / 1000
# condense condense_groups
nb = get_condense_sum(nb, c1_groups, c1_groups_name)
# rename unhandy column names
nb.rename(columns=carrier_renaming, inplace=True)

nb[nb > 0].sum()[index].sum() / 1e3
# STST: 3102 TWh
# EXP: 2420 TWh

# H2 is only the bus, so it has to be exluded when calculating the production / consumption (it is 0 in total)
# is this the amount of hydrogen that is going through the pipeline?

In [ ]:
nb[nb > 0].sum()[index] / 1e3

In [ ]:
df_stst_ons[[c + "_gen_h2" for c in c_h2_gen]].sum()

In [ ]:
# production
df_stst_ons[[c + "_gen_h2" for c in c_h2_gen]].sum().sum() /1e3
df_exp_ons[[c + "_gen_h2" for c in c_h2_gen]].sum().sum() /1e3

# STST: 3102 TWh
# EXP: 2420 TWh

In [ ]:
df_stst_ons[[c + "_gen_h2" for c in c_h2_gen]].sum() /1e3

In [ ]:
df_exp_ons[[c + "_gen_h2" for c in c_h2_gen]].sum() /1e3

In [ ]:
df_stst_ons[[c + "_gen_h2" for c in c_h2_gen]].sum()["SMR_gen_h2"]# / df_stst_ons[[c + "_gen_h2" for c in c_h2_gen]].sum().sum()

In [ ]:
# consumption
df_stst_ons[[c + "_con_h2" for c in c_h2_con]].sum().sum() /1e3
df_exp_ons[[c + "_con_h2" for c in c_h2_con]].sum().sum() /1e3

# STST: 2631 TWh
# EXP: 1949 TWh

In [ ]:
df_stst_ons[[c + "_con_h2" for c in c_h2_con]].sum() /1e3

## Production and Consumption correlation (spatial & temporal)

In [ ]:
index = [c + "_gen_el" for c in c_el_gen_s] + [c + "_con_el" for c in c_el_con_s] + [c + "_gen_h2" for c in c_h2_gen] + [c + "_con_h2" for c in c_h2_con]
cols = [carrier_renaming.get(n, n) for n in c_el_gen_s] + [carrier_renaming.get(n, n) + " (con)" for n in c_el_con_s] + [carrier_renaming.get(n, n) for n in c_h2_gen] + [carrier_renaming.get(n, n) + " (con)" for n in c_h2_con]

In [ ]:
# Spatial correlation of gen STST and EXP
# correlation of the whole production and consumption aggregated for all locations

fig, axs = plt.subplots(ncols=2, figsize=(22, 8))

for i, df, ax in zip([0,1], [df_stst_ons, df_exp_ons], axs):
    corr = df[index].corr()
    corr.index = cols
    corr.columns = cols

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.4], mask=mask, cmap="magma_r", annot=True, ax=ax)
    ax.set_title('STST' if i == 0 else 'EXP')

fig.tight_layout(pad=1.5)
#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}01_general/4.1_system_overview/el_gen_con_spatial_corr.png")

In [ ]:
df_stst_ts

In [ ]:
# temporal correlation of gen STST and EXP (1)
# correlation of the production / consumption per time step aggregated over the locations

fig, axs = plt.subplots(ncols=2, figsize=(22, 8))

for i, df, ax in zip([0,1], [df_stst_ts, df_exp_ts], axs):
    corr = df[index].corr()
    corr.index = cols
    corr.columns = cols

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.4], mask=mask, cmap="magma_r", annot=True, ax=ax)
    ax.set_title('STST' if i == 0 else 'EXP')


fig.tight_layout(pad=1.5)
#plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}01_general/4.1_system_overview/el_gen_con_temporal_corr.png")

In [ ]:
# thesis plot
# Spatial and temporal correlation of gen STST and EXP

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(22, 18))

for title, df, ax in zip(["STST (spatial)", "EXP (spatial)", "STST (temporal)", "EXP (temporal)"], [df_stst_ons, df_exp_ons, df_stst_ts, df_exp_ts], axs.reshape(-1)):
    corr = df[index].corr()
    corr.index = cols
    corr.columns = cols

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.4], mask=mask, cmap="magma_r", annot=True, ax=ax)
    ax.hlines(len(index) - 7, 0, len(index), colors="red", ls=":")
    ax.xaxis.set_tick_params(labelsize=14)
    ax.yaxis.set_tick_params(labelsize=14)
    ax.set_title(title, fontsize=23, **font1)

fig.tight_layout(pad=1)
#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}/h2_gen_con_spatial_temporal_corr.png")